### Imports

In [1]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential, load_model
from keras.layers import Dense, Bidirectional
from keras.layers.recurrent import GRU

from encrypt import *
from tools import *
import string

Using TensorFlow backend.


### Configurations

In [2]:
letters = string.printable.split('!')[0]
encrypt = shifted_vigenere_cipher
samples_count = 200000
text_length = 10

set_characters(letters)
model_path = f"models/{encrypt.__name__}_{len(letters)}x{text_length}_best_model.h5"

### Generating and Preparing Data

In [3]:
text = generate_text(text_length, samples_count)
cipher = list(map(encrypt, text))

train_text = to_vec(text)
train_cipher = to_vec(cipher)

### Building and Training the Model

In [4]:
model = Sequential()
model.add(Bidirectional(GRU(64, activation='relu', return_sequences=True), input_shape=train_cipher.shape[1:]))
model.add(Dense(1))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 10, 128)           25344     
_________________________________________________________________
dense_1 (Dense)              (None, 10, 1)             129       
Total params: 25,473
Trainable params: 25,473
Non-trainable params: 0
_________________________________________________________________


In [5]:
callbacks = [
    EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=3, verbose=1, mode='min'),
    ModelCheckpoint(model_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
]
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.fit(train_cipher, train_text, validation_split=.1, batch_size=10, epochs=10, callbacks=callbacks)

Train on 180000 samples, validate on 20000 samples
Epoch 1/10
180000/180000 [==============================] - 115s 640us/step - loss: 27.1789 - acc: 0.2557 - val_loss: 1.3223 - val_acc: 0.4839

Epoch 00001: val_acc improved from -inf to 0.48393, saving model to models/shifted_vigenere_cipher_62x10_best_model.h5
Epoch 2/10
180000/180000 [==============================] - 116s 645us/step - loss: 2.3870 - acc: 0.5080 - val_loss: 1.0974 - val_acc: 0.5592

Epoch 00002: val_acc improved from 0.48393 to 0.55923, saving model to models/shifted_vigenere_cipher_62x10_best_model.h5
Epoch 3/10
180000/180000 [==============================] - 117s 651us/step - loss: 1.3536 - acc: 0.6661 - val_loss: 0.2097 - val_acc: 0.7917

Epoch 00003: val_acc improved from 0.55923 to 0.79167, saving model to models/shifted_vigenere_cipher_62x10_best_model.h5
Epoch 4/10
180000/180000 [==============================] - 117s 653us/step - loss: 0.8169 - acc: 0.7692 - val_loss: 0.1487 - val_acc: 0.8755

Epoch 00004: 

### Predicting and Evaluating the Model

In [6]:
model = load_model(model_path)

In [7]:
test_cyhper = encrypt('HelloWorld')
prediction = to_txt(model.predict(to_vec(test_cyhper)))

print('Model predicted that original text is:', prediction)
print('Test cipher vs encrypted prediction:', test_cyhper, 'vs', encrypt(prediction))

Model predicted that original text is: HelloWorld
Test cipher vs encrypted prediction: 2vzunIgopt vs 2vzunIgopt
